# Test cTraderFixPy

https://github.com/spotware/cTraderFixPy

https://spotware.github.io/cTraderFixPy/

In [ ]:
from twisted.internet import reactor
from inputimeout import inputimeout, TimeoutOccurred
import json
from ctrader_fix import *

from configs.settings import work_dir

In [ ]:
# you can use two separate config files for QUOTE and TRADE
config_type = (
    "trade"
    # "quote"
)
with open(f"{work_dir()}/src/configs/ctrader_{config_type}_demo_1.json") as configFile:
    config = json.load(configFile)
config

In [ ]:
# set up client based on the config
client = Client(config["Host"], config["Port"], ssl = config["SSL"])
client

In [ ]:
# Callback for receiving all messages
# def onMessageReceived(client, responseMessage):
#    print("Received: ", responseMessage.getMessage().replace("\01", "|"))
#    messageType = responseMessage.getFieldValue(35)
#    if messageType == "A":
#        print("We are logged in")

In [ ]:
def send(request):
    diferred = client.send(request)
    diferred.addCallback(lambda _: print("\nSent: ", request.getMessage(client.getMessageSequenceNumber()).replace("\01", "|")))
    # diferred.addCallback(lambda _: print("\nSent: ", request.getMessage(client.getMessageSequenceNumber())))



In [ ]:
def onMessageReceived(client, responseMessage): # Callback for receiving all messages
    print("\nReceived: ", responseMessage.getMessage().replace("\01", "|"))
    # We get the message type field value
    messageType = responseMessage.getFieldValue(35)
    a = responseMessage.getFieldValue(721)
    print(f"a={a}")
    # we send a security list request after we received logon message response
    if messageType == "A":
        print("A")
        securityListRequest = SecurityListRequest(config)
        securityListRequest.SecurityReqID = "A"
        securityListRequest.SecurityListRequestType = 0
        print(f"securityListRequest={securityListRequest}")
        send(securityListRequest)
    # After receiving the security list we send a market order request by using the security list first symbol
    elif messageType == "y":
        print("B")
        # We use getFieldValue to get all symbol IDs, it will return a list in this case
        # because the symbol ID field is repetitive
        symboldIds = responseMessage.getFieldValue(55)
        print(f"symboldIds={symboldIds}")
        print(f"symboldIds[1]={symboldIds[2]}")
        print("C")
        if config["TargetSubID"] == "TRADE":
            
            print("D")
            newOrderSingle = NewOrderSingle(config)
            newOrderSingle.ClOrdID = "B"
            # newOrderSingle.Symbol = symboldIds[1]
            newOrderSingle.Symbol = "41" # AUDCHF
            newOrderSingle.Side = 2
            newOrderSingle.OrderQty = 2
            newOrderSingle.OrdType = 1
            # newOrderSingle.Price = 1940
            # newOrderSingle.StopPx = 2000
            #newOrderSingle.OrdType = 2  # Limit order
            #newOrderSingle.Price = 1950.00  # Set the buy price here
            newOrderSingle.Designation = "From Jupyter"
            print(f"newOrderSingle={newOrderSingle}")
            send(newOrderSingle)
            print("E")
            
            print("D2")
            newOrderSingle = NewOrderSingle(config)
            newOrderSingle.ClOrdID = "B2"
            # newOrderSingle.Symbol = symboldIds[1]
            newOrderSingle.Symbol = "41" # AUDCHF
            newOrderSingle.Side = 2
            newOrderSingle.OrderQty = 3
            newOrderSingle.OrdType = 1
            # newOrderSingle.Price = 1940
            # newOrderSingle.StopPx = 2000
            #newOrderSingle.OrdType = 2  # Limit order
            #newOrderSingle.Price = 1950.00  # Set the buy price here
            newOrderSingle.Designation = "From Jupyter"
            print(f"newOrderSingle={newOrderSingle}")
            send(newOrderSingle)
            print("E2")
            
            print("ZZZZ REquest for postions")
            r=RequestForPositions(config)
            r.PosReqID=338960103
            print(f"newOrderSingle={newOrderSingle}")
            send(r)
            print("r done")
            
            
        else:
            print("F")
            marketDataRequest = MarketDataRequest(config)
            marketDataRequest.MDReqID = "a"
            marketDataRequest.SubscriptionRequestType = 1
            marketDataRequest.MarketDepth = 1
            marketDataRequest.NoMDEntryTypes = 1
            marketDataRequest.MDEntryType = 0
            marketDataRequest.NoRelatedSym = 1
            # marketDataRequest.Symbol = symboldIds[1]
            marketDataRequest.Symbol = "41"
            send(marketDataRequest)
            print(f"marketDataRequest={marketDataRequest}")
            print("G")
    # after receiving the new order request response we stop the reactor
    # And we will be disconnected from API
    elif messageType == "8" or messageType == "j":
        print("H")
        print("We are done, stopping the reactor")
        # reactor.stop()
    print("I")


In [ ]:
# Callback for client disconnection
def disconnected(client, reason): 
    print("Disconnected, reason: ", reason)

In [ ]:
# Callback for client connection
def connected(client):
    print("Connected")
    logonRequest = LogonRequest(config)
    send(logonRequest)

In [ ]:
# Setting client callbacks
client.setConnectedCallback(connected)
client.setDisconnectedCallback(disconnected)
client.setMessageReceivedCallback(onMessageReceived)

In [ ]:
# Starting the client service
client.startService()

In [ ]:
# run
reactor.run()

In [ ]:
a